# Python Client

### Import ModelBazaar class.

In [ ]:
import thirdai
from thirdai.neural_db import ModelBazaar

# Activate your license
thirdai.licensing.activate("YOUR-THIRDAI-ACTIVATION-KEY")

### Initialise a ModelBazaar object with url where model_bazaar is hosted.

In [ ]:
# default port is 80
bazaar = ModelBazaar(base_url="http://<bazaar-endpoint>/api/")

### Sign up

In [ ]:
# Sends a email verification link.
bazaar.sign_up(email="mritunjay@thirdai.com", password="password", username="mj3ai")

### Login

In [ ]:
bazaar.log_in(email="mritunjay@thirdai.com", password="password")

### Train options

In [ ]:
train_extra_options = {
    "num_samples_per_shard": 3500000,
    "allocation_memory": 250000,
#   ---shard agnostic training params---
    "model_cores": 2,
    "model_memory": 200000,
    "csv_id_column": "label",
    "csv_strong_columns": ["title"],
    "csv_weak_columns": ["abstract"],
    "csv_reference_columns": ["title", "abstract"],
    "fhr": 200000,
    "embedding_dim": 4096,
    "output_dim": 100000,
    "max_in_memory_batches": 200
}

### Creates a model from scratch trained on given list of documents.

In [ ]:
# Can be synchronous or asynchronous(default; in which case we call await_train)
# `doc_type` can be "local"(default), "nfs" or "s3".
model = bazaar.train(
    model_name="pubmed-35M-10-models-char4-0",
    docs=["/model_bazaar/datasets/pubmed/lowercased_all_withoutpapertags.csv"],
    doc_type="nfs",
    sharded=True,
    is_async=True,
    train_extra_options=train_extra_options,
)

### Blocking call to wait till model finishes training.

In [ ]:
bazaar.await_train(model)

### Deploys the model and returns an ndb_client which can be used to interact to neural_db.

In [ ]:
# Can be synchronous or asynchronous(default; in which case we call await_deploy)
ndb_client = bazaar.deploy(
    model_identifier=model.model_identifier,
    deployment_name="deployment-0",
    is_async=True,
)

### Blocking call to wait till deployment finishes.

In [ ]:
bazaar.await_deploy(ndb_client)

### Insert new files in the neural_db model.

In [ ]:
ndb_client.insert(
    files=["/Users/mjay/Documents/MACH.pdf", "/Users/mjay/Documents/OpenMPIInstall.pdf"]
)

### Search the ndb model.

In [ ]:
results = ndb_client.search(query="who are the authors of this paper", top_k="5")

query_text = results["query_text"]
references = results["references"]
for reference in references:
    print(reference["text"])

### [RLHF] Associate : takes list of dictionaries where each dictionary has 'source' and 'target' keys.

In [ ]:
ndb_client.associate(
    [
        {"source": "authors", "target": "contributors"},
        {"source": "paper", "target": "document"},
    ]
)

### [RLHF] Upvote/Downvote : takes list of dictionaries where each dictionary has 'query_text' and 'reference_id' keys.

In [ ]:
best_answer = references[4]
good_answer = references[2]
ndb_client.upvote(
    [
        {"query_text": query_text, "reference_id": best_answer["id"]},
        {"query_text": query_text, "reference_id": good_answer["id"]},
    ]
)

### Undeploys the model.

In [ ]:
bazaar.undeploy(ndb_client)

### Deletes the model from model bazaar.

In [ ]:
bazaar.delete(model_identifier=model.model_identifier)

### Return a list of models accessible to user on model bazaar.

In [ ]:
bazaar.list_models()

### Returns a list of all active deployments.

In [ ]:
bazaar.list_deployments()

### Connects to an active deployment.

In [ ]:
ndb_client = bazaar.connect(deployment_identifier="mj3ai/model-0:mj3ai/deployment-0")

### Push an NDB model from local to model bazaar

In [ ]:
bazaar.push_model(
    model_name="test-upload",
    local_path="model_bazaar_cache/mj3ai/test-model-4/model.ndb",
    access_level="private",
)

### Pull model from model bazaar to local

In [ ]:
ndb_model = bazaar.pull_model(model_identifier="mj3ai/model-1")